# Sentiment Analysis Model training

In [65]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [66]:
from string import punctuation
from nltk.corpus import stopwords
print(stopwords.words('english')[10:15])

def punctuation_stopwords_removal(sms):
    # filters charecter-by-charecter : ['h', 'e', 'e', 'l', 'o', 'o', ' ', 'm', 'y', ' ', 'n', 'a', 'm', 'e', ' ', 'i', 's', ' ', 'p', 'u', 'r', 'v', 'a']
    remove_punctuation = [ch for ch in sms if ch not in punctuation]
    # convert them back to sentences and split into words
    remove_punctuation = "".join(remove_punctuation).split()
    filtered_sms = [word.lower() for word in remove_punctuation if word.lower() not in stopwords.words('english')]
    return filtered_sms

["you've", "you'll", "you'd", 'your', 'yours']


In [67]:
sentiment_df = pd.read_csv('/kaggle/input/twitterdata/finalSentimentdata2.csv')

In [68]:
sentiment_df.loc[:, 'text'] = sentiment_df['text'].apply(punctuation_stopwords_removal)

In [69]:
reviews_split = []
for i, j in sentiment_df.iterrows():
    reviews_split.append(j['text'])

In [70]:
words = []
for review in reviews_split:
    for word in review:
        words.append(word)


## Encoding 

In [71]:
from collections import Counter

counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word:ii for ii, word in enumerate(vocab, 1)}

In [72]:
encoded_reviews = []
for review in reviews_split:
    encoded_reviews.append([vocab_to_int[word] for word in review])


In [73]:
print(len(vocab_to_int))
print(encoded_reviews[:10])

10662
[[853, 186, 20, 1079, 1457, 4429, 2201, 407, 1240, 1079, 15, 218, 337, 167, 253, 462, 337, 122, 168, 4430, 4431, 140, 23, 264, 58, 765, 3, 5, 195, 1079, 2966, 274], [80, 1755, 4432, 2967, 4433, 86, 854, 1080, 2968, 4434, 4435, 7], [543, 4436, 946, 1458, 265, 2, 1241, 168, 2202], [1, 4437, 169, 266, 1459, 129, 1242, 47, 7], [304, 1756, 4438, 168, 8, 39, 219, 93, 355, 4, 21], [2203, 2204, 1, 1757, 1243, 2969, 1460, 1081, 1461, 4439, 98, 4440], [947, 37, 1758, 285, 948, 4441, 1462, 2205, 13, 3, 5, 4442, 285, 1244, 4443, 1463, 4444, 4445, 286, 4446, 15, 4447, 47, 228, 2970, 338, 40, 312, 1463, 179, 1759], [41, 377, 149, 1245, 4448, 34], [2206, 649, 180, 1760, 2207, 91, 650, 378, 463, 1246, 595, 1464, 2208, 2, 8, 2209, 651, 40, 379, 2210, 21, 228, 703, 1246, 1761, 408], [2206, 649, 180, 1760, 2207, 91, 650, 378, 463, 1246, 595, 1464, 2208, 2, 8, 2209, 651, 40, 379, 2210, 21, 228, 703, 1246, 2971]]


In [74]:
labels_to_int = []
for i, j in sentiment_df.iterrows():
    if j['sentiment']=='joy':
        labels_to_int.append(1)
    else:
        labels_to_int.append(0)
    

## Detecting any outlier reviews

In [75]:
reviews_len = Counter([len(x) for x in encoded_reviews])
print(max(reviews_len))

48


In [76]:
print(len(encoded_reviews))

3090


In [77]:
non_zero_idx = [ii for ii, review in enumerate(encoded_reviews) if len(encoded_reviews)!=0]
encoded_reviews = [encoded_reviews[ii] for ii in non_zero_idx]
encoded_labels = np.array([labels_to_int[ii] for ii in non_zero_idx])

In [78]:
print(len(encoded_reviews))
print(len(encoded_labels))

3090
3090


In [79]:
def pad_features(reviews_int, seq_length):
    features = np.zeros((len(reviews_int), seq_length), dtype=int)
    for i, row in enumerate(reviews_int):
        if len(row)!=0:
            features[i, -len(row):] = np.array(row)[:seq_length]
    return features

In [80]:
seq_length = 50
padded_features= pad_features(encoded_reviews, seq_length)
print(padded_features[:2])


[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0  853  186   20 1079 1457 4429 2201  407 1240 1079
    15  218  337  167  253  462  337  122  168 4430 4431  140   23  264
    58  765    3    5  195 1079 2966  274]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0   80 1755 4432 2967
  4433   86  854 1080 2968 4434 4435    7]]


## Training, Testing and Validating

In [81]:
split_frac = 0.8
split_idx = int(len(padded_features)*split_frac)

training_x, remaining_x = padded_features[:split_idx], padded_features[split_idx:]
training_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]


## Dataloaders and Batching

In [82]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [83]:
# torch.from_numpy creates a tensor data from n-d array
train_data = TensorDataset(torch.from_numpy(training_x), torch.from_numpy(training_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))

batch_size = 1

train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)
valid_loader = DataLoader(valid_data, batch_size=batch_size)

In [84]:
gpu_available = torch.cuda.is_available

if gpu_available:
    print('Training on GPU')
else:
    print('GPU not available')

Training on GPU


## Sentiment Network with PyTorch

In [85]:
import torch.nn as nn

class CovidTweetSentimentAnalysis(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.2):
        super(CovidTweetSentimentAnalysis, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
    
    def forward(self, x, hidden):
        # x : batch_size * seq_length * features
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding_layer(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        sig_out = self.sig(out)
        
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]
        
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        # initialize weights for lstm layer
        weights = next(self.parameters()).data
        
        if gpu_available:
            hidden = (weights.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                     weights.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weights.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                     weights.new(self.n_layers, batch_size, self.hidden_dim).zero())
        return hidden

In [86]:
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1 # either happy or sad
embedding_dim = 400
hidden_dim = 256
n_layers = 2

In [87]:
net = CovidTweetSentimentAnalysis(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net)

CovidTweetSentimentAnalysis(
  (embedding_layer): Embedding(10663, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [88]:
lr = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [89]:
epochs = 4
count = 0
print_every = 100
clip = 5 
if gpu_available:
    net.cuda()

net.train()
for e in range(epochs):
    # initialize lstm's hidden layer 
    h = net.init_hidden(batch_size)
    for inputs, labels in train_loader:
        count += 1
        if gpu_available:
            inputs, labels = inputs.cuda(), labels.cuda()
        h = tuple([each.data for each in h])
        
        # training process
        net.zero_grad()
        outputs, h = net(inputs, h)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm(net.parameters(), clip)
        optimizer.step()
        
        # print average training losses
        if count % print_every == 0:
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:
                val_h = tuple([each.data for each in val_h])
                if gpu_available:
                    inputs, labels = inputs.cuda(), labels.cuda()
            outputs, val_h = net(inputs, val_h)
            val_loss = criterion(outputs.squeeze(), labels.float())
            val_losses.append(val_loss.item())
        
            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(count),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch: 1/4... Step: 100... Loss: 0.098594... Val Loss: 0.235060
Epoch: 1/4... Step: 200... Loss: 0.041589... Val Loss: 0.234342
Epoch: 1/4... Step: 300... Loss: 0.058962... Val Loss: 0.163241
Epoch: 1/4... Step: 400... Loss: 0.140679... Val Loss: 0.296221
Epoch: 1/4... Step: 500... Loss: 1.787473... Val Loss: 0.532953
Epoch: 1/4... Step: 600... Loss: 0.131555... Val Loss: 0.479034
Epoch: 1/4... Step: 700... Loss: 1.579360... Val Loss: 0.357029
Epoch: 1/4... Step: 800... Loss: 0.427348... Val Loss: 1.242114
Epoch: 1/4... Step: 900... Loss: 0.927642... Val Loss: 1.237466
Epoch: 1/4... Step: 1000... Loss: 0.034962... Val Loss: 0.713713
Epoch: 1/4... Step: 1100... Loss: 0.079024... Val Loss: 0.365033
Epoch: 1/4... Step: 1200... Loss: 0.686994... Val Loss: 0.690902
Epoch: 1/4... Step: 1300... Loss: 0.018469... Val Loss: 0.690672
Epoch: 1/4... Step: 1400... Loss: 0.039891... Val Loss: 0.804481
Epoch: 1/4... Step: 1500... Loss: 1.237151... Val Loss: 1.526448
Epoch: 1/4... Step: 1600... Loss: 

In [90]:
torch.save(net.state_dict(), 'model.pt')

In [91]:
from string import punctuation

def tokenize_covid_tweet(tweet):
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in tweet])
    return test_ints

In [92]:
def predict_covid_sentiment(net, test_tweet, seq_length=50):
    print('Original Sentence :')
    print(test_tweet)
    
    print('\nAfter removing punctuations and stop-words :')
    test_tweet = punctuation_stopwords_removal(test_tweet)
    print(test_tweet)
    
    print('\nAfter converting pre-processed tweet to tokens :')
    tokenized_tweet = tokenize_covid_tweet(test_tweet)
    print(tokenized_tweet)
    
    print('\nAfter padding the tokens into fixed sequence lengths :')
    padded_tweet = pad_features(tokenized_tweet, 50)
    print(padded_tweet)
    
    feature_tensor = torch.from_numpy(padded_tweet)
    batch_size = feature_tensor.size(0)
    
    if gpu_available:
        feature_tensor = feature_tensor.cuda()
    
    h = net.init_hidden(batch_size)
    output, h = net(feature_tensor, h)
    
    predicted_sentiment = torch.round(output.squeeze())
    print('\n==========Predicted Sentiment==========\n')
    if predicted_sentiment == 1:
        print('Happy')
    else:
        print('Sad')
    print('\n==========Predicted Sentiment==========\n')


In [93]:
test_sad_tweet = input()
predict_covid_sentiment(net, test_sad_tweet)

 It is very sad to see the corona pandemic increasing at such an alarming rate


Original Sentence :
It is very sad to see the corona pandemic increasing at such an alarming rate

After removing punctuations and stop-words :
['sad', 'see', 'corona', 'pandemic', 'increasing', 'alarming', 'rate']

After converting pre-processed tweet to tokens :
[[328, 63, 2, 28, 1964, 6137, 267]]

After padding the tokens into fixed sequence lengths :
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0  328   63    2   28 1964 6137  267]]

==========Predicted Sentiment==========

Sad

==========Predicted Sentiment==========

